<a href="https://colab.research.google.com/github/Meenusj/deepfake-detection/blob/main/fasttextfordeepfake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 499.3 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199775 sha256=c4d1f9de2b4d7ef34096901cac93d2a6963586cf5c3b46c92e7860e37bf702cf
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
!gzip -d cc.en.300.bin.gz

--2024-01-10 06:16:59--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.162.163.34, 3.162.163.19, 3.162.163.11, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.162.163.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘cc.en.300.bin.gz’

cc.en.300.bin.gz    100%[===================>]   4.19G   208MB/s    in 30s     

2024-01-10 06:17:29 (144 MB/s) - ‘cc.en.300.bin.gz’ saved [4503593528/4503593528]



In [1]:
import pandas as pd
import numpy as np
import fasttext
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import pickle

# Load TweepFake dataset (replace 'your_dataset.csv' with the actual file path)
df = pd.read_csv('train.csv')

# Assuming your dataset has a 'status_id' column containing tweet text and a 'class_type' column with class labels
texts = df['status_id'].astype(str)
labels = df['class_type']

# Load the pre-trained FastText model (replace 'cc.en.300.bin' with the path to your model)
model = fasttext.load_model('cc.en.300.bin')

# Convert each tweet text to a vector
def text_to_vector(text):
    words = text.split()
    vectors = [model.get_word_vector(word) for word in words]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.get_dimension())

# Apply the text_to_vector function to each tweet in the dataset
df['text_vector'] = texts.apply(text_to_vector)

# Save the resulting vectors to a new CSV file
df.to_csv('word_vectors.csv', index=False)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    df['text_vector'].to_list(),
    labels,
    test_size=0.2,
    random_state=42
)

# Convert X_train and X_test to numpy arrays
X_train = np.array(X_train)
X_test = np.array(X_test)

# Train a logistic regression classifier
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

classifier = LogisticRegression(solver='liblinear', max_iter=1000)
classifier.fit(X_train_scaled, y_train)

# Save the classifier and scaler using pickle
with open('classifier_model.pkl', 'wb') as model_file:
    pickle.dump(classifier, model_file)

with open('scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

# Make predictions on the test set
predictions = classifier.predict(X_test_scaled)

# Print the predictions
# print("Predicted Labels:", predictions)
# Make predictions on the test set
predictions = classifier.predict(X_test_scaled)

# Print actual and predicted labels
result_df = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})
print(result_df)


# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy}')


ModuleNotFoundError: No module named 'fasttext'